In [3]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle


In [4]:
from zipfile import ZipFile
!unzip rice_disease.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
# Load the dataset
data_dir = 'Rice_Leaf_AUG'  # Change to your dataset path
categories = os.listdir(data_dir)
images = []
labels = []

for category in categories:
    folder_path = os.path.join(data_dir, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (150, 150))  # Resize images to 128x128
        images.append(img)
        labels.append(category)

images = np.array(images)
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train = X_train / 255.0
X_test = X_test / 255.0


In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
feature_extractor = Model(inputs=base_model.input, outputs=x)


In [12]:
# Extract features
def extract_features(images):
    features = feature_extractor.predict(images)
    return features

train_features = extract_features(X_train)
test_features = extract_features(X_test)

24/24 [==============================] - 49s 2s/step


In [14]:
# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, y_train)

y_pred = rf_model.predict(test_features)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Random Forest Accuracy: 0.7689295039164491
                       precision    recall  f1-score   support

Bacterial Leaf Blight       0.86      0.77      0.81       147
           Brown Spot       0.72      0.76      0.74       123
    Healthy Rice Leaf       0.90      0.85      0.88       130
           Leaf Blast       0.77      0.69      0.73       131
           Leaf scald       0.71      0.72      0.72       115
        Sheath Blight       0.67      0.82      0.74       120

             accuracy                           0.77       766
            macro avg       0.77      0.77      0.77       766
         weighted avg       0.78      0.77      0.77       766



In [16]:
os.makedirs("models", exist_ok=True)
feature_extractor.save(os.path.join("models", 'cnn_model.h5'))
with open(os.path.join("models", 'rf_model.pkl'), 'wb') as f:
    pickle.dump(rf_model, f)

print("Models saved successfully!")

Models saved successfully!


c:\Users\noorh\Documents\Bytewise\rice_disease_detection\menv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**CNN MODEL FOR CLASSIFICATION**

In [18]:
# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

# Train the model
cnn_model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Save the CNN model
cnn_model.save('CNN_MODEL.h5')


Epoch 1/10
96/96 [==============================] - 64s 647ms/step - loss: 1.7740 - accuracy: 0.2488 - val_loss: 1.6075 - val_accuracy: 0.3290
Epoch 2/10
96/96 [==============================] - 57s 592ms/step - loss: 1.6009 - accuracy: 0.3183 - val_loss: 1.4620 - val_accuracy: 0.3851
Epoch 3/10
96/96 [==============================] - 58s 601ms/step - loss: 1.5004 - accuracy: 0.3817 - val_loss: 1.2610 - val_accuracy: 0.5209
Epoch 4/10
96/96 [==============================] - 60s 622ms/step - loss: 1.3756 - accuracy: 0.4600 - val_loss: 1.1193 - val_accuracy: 0.5796
Epoch 5/10
96/96 [==============================] - 60s 623ms/step - loss: 1.2818 - accuracy: 0.4989 - val_loss: 1.0650 - val_accuracy: 0.6162
Epoch 6/10
96/96 [==============================] - 61s 638ms/step - loss: 1.2127 - accuracy: 0.5260 - val_loss: 0.9645 - val_accuracy: 0.6436
Epoch 7/10
96/96 [==============================] - 70s 730ms/step - loss: 1.2076 - accuracy: 0.5446 - val_loss: 0.9671 - val_accuracy: 0.6580